In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import datetime
import time
import math
from sklearn.preprocessing import MinMaxScaler

In [0]:
campaign_data = pd.read_csv("campaign_data.csv")
campaign_data.head(1)

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13


In [0]:
coupon_item_mapping = pd.read_csv("coupon_item_mapping.csv")
coupon_item_mapping.head(2)

,coupon_id,item_id
0,105,37
1,107,75


In [0]:
customer_demographics = pd.read_csv("customer_demographics.csv")
customer_demographics.head(2)

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5


In [0]:
customer_transaction_data = pd.read_csv("customer_transaction_data.csv")
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [0]:
item_data = pd.read_csv("item_data.csv")
item_data.head(2)

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous


In [0]:
train = pd.read_csv("train.csv")
train.head(2)

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0


In [0]:
test = pd.read_csv("test_QyjYwdj.csv")
test.head(2)

,id,campaign_id,coupon_id,customer_id
0,3,22,869,967
1,4,20,389,1566


In [0]:
def function(train1, campaign_data, customer_demographics, customer_transaction_data, item_data, coupon_item_mapping): 
  #1. Merge train data with campaign data on campaign_id
  df1 = pd.merge(train1, campaign_data, on = "campaign_id", how = "left")
  #2. Merge 1 with customer demographic data on customer_id
  df2 = pd.merge(df1, customer_demographics, on = "customer_id", how = "left") 
  #3. Merge coupon-item data with item data on item_id
  df3 = pd.merge(coupon_item_mapping, item_data, on = "item_id")
  #4. Extract aggregated features one for each coupon_id from 3
  df4 = df3.groupby("coupon_id").aggregate({"brand":["nunique"], 
                "brand_type":["nunique"], 
                "category":["nunique"]}).reset_index()
  #5. Merge 2 and 4 on coupon_id
  df5 = pd.merge(df2, df4, on = "coupon_id", how = "left")
  #6. Merge transaction data with item data and customer demographics data on item_id and customer_id respectively
  df6 = pd.merge(customer_transaction_data, item_data, on = "item_id", how = "left")
  df6 = pd.merge(df6, customer_demographics, on = "customer_id", how = "left")
  df7 = df6.groupby("customer_id").aggregate({"item_id":["nunique"], 
                "quantity":["sum"], 
                "selling_price":["median"],
                "other_discount":["median"],
                "coupon_discount":["median"],
                "brand":["nunique"],
                "brand_type":["nunique"],
                "category":["nunique"]
                }).reset_index()
  df7.columns = df7.columns.droplevel(1)
  df_final = pd.merge(df5, df7, on = "customer_id", how = "left")
  df_final.drop(["brand", "brand_type", "category"], axis = 1, inplace = True)
  train1 = df_final
  df = pd.get_dummies(train1["age_range"]).drop("70+", axis = 1)
  train1 = train1.join(df)
  train1.drop("age_range", axis = 1, inplace = True)
  df1 = pd.get_dummies(train1["campaign_type"]).drop("Y", axis = 1)
  train1 = train1.join(df1)
  train1.drop("campaign_type", axis = 1, inplace = True)
  train1['start_date'] = train1['start_date'].apply(lambda x: x.replace("/", ","))
  train1['end_date'] = train1['end_date'].apply(lambda x: x.replace("/", ","))
  train1['start_date'] = pd.to_datetime(train1['start_date'], format='%d,%m,%y').dt.strftime('%Y,%m,%d')
  train1['end_date'] = pd.to_datetime(train1['end_date'], format='%d,%m,%y').dt.strftime('%Y,%m,%d')
  train1["start_date"] = train1["start_date"].apply(lambda x: time.mktime(time.strptime(x, "%Y,%m,%d")))/(60*60*24)
  train1["end_date"] = train1["end_date"].apply(lambda x: time.mktime(time.strptime(x, "%Y,%m,%d")))/(60*60*24)
  train1["duration"] = train1["end_date"] - train1["start_date"]
  train1.drop(["end_date", "start_date"], axis = 1, inplace = True)
  train1["income_bracket"].fillna(train1["income_bracket"].mean(), inplace = True)
  train1["rented"].fillna(0, inplace = True)
  train1["family_size"].fillna(1, inplace = True)
  train1["family_size"].replace(['1', '2', 1, '3', '4', '5+'], [1, 2, 1, 3, 4, 5], inplace = True)
  train1["marital_status"].fillna(0, inplace = True)
  t_df = train1[train1["marital_status"] == 0]
  t_df0 = t_df[t_df["family_size"] == 1]
  t_df0["marital_status"].replace(0, "Single", inplace = True)
  t_df1 = t_df[t_df["family_size"] != 1]
  t_df1["marital_status"].replace(0, "Married", inplace = True)
  t_df = t_df0.append(t_df1)
  t_df2 = train1[train1["marital_status"] != 0]    
  train1 = t_df.append(t_df2)
  married = pd.get_dummies(train1["marital_status"]).drop("Single", axis = 1)
  train1 = train1.join(married)
  train1.drop("marital_status", axis = 1, inplace = True)
  train1.sort_values(by = "id", inplace = True)
  train1["no_of_children"] = train1["family_size"] - 2
  train1["no_of_children"].replace(-1, 0, inplace = True)
  train1["income_bracket"] = train1["income_bracket"].apply(lambda x: round(x))
  family = pd.get_dummies(train1["family_size"]).add_prefix("f")
  child = pd.get_dummies(train1["no_of_children"]).add_prefix("c")
  income = pd.get_dummies(train1["income_bracket"]).add_prefix("i")
  train1 = train1.join(family)
  train1 = train1.join(child)
  train1 = train1.join(income)
  train1["rented"] = train1["rented"].astype(int)
  train1.drop(["income_bracket", "no_of_children", "family_size"], inplace = True, axis = 1)
  train1["total_discount"] = (abs(train1["other_discount"])+abs(train1["coupon_discount"]))/(train1["selling_price"] - train1["other_discount"]-train1["coupon_discount"])
  #train1["discount_ratio"] = train1["coupon_discount"]/train1["other_discount"]
  #train1.drop(["other_discount", "coupon_discount"], inplace = True, axis = 1)
  scaler = MinMaxScaler()
  train1["quantity"] = scaler.fit_transform(train1["quantity"].values.reshape(-1,1))
  train1["duration"] = scaler.fit_transform(train1["duration"].values.reshape(-1,1))
  #train1.drop("selling_price", axis = 1, inplace = True)
  #quantity  duration  total_discount  discount_ratio
  train1["quantity"] = train1["quantity"]*10
  train1["duration"] = train1["duration"]*10
  train1["total_discount"] = train1["total_discount"]*10
  # train1["discount_ratio"] = train1["discount_ratio"]*10
  train1.drop(["f5", "c3", "i12"], axis = 1, inplace = True)
  # train1.drop("quantity", inplace = True, axis = 1)
  train1["price_per_qty"] = train1["selling_price"]/train1["quantity"]
  return train1

In [0]:
train = function(train, campaign_data, customer_demographics, customer_transaction_data, item_data, coupon_item_mapping)


/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [0]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,rented,income_bracket,"(brand, nunique)","(brand_type, nunique)","(category, nunique)",item_id,quantity,selling_price,other_discount,coupon_discount,18-25,26-35,36-45,46-55,56-70,X,duration,Married,f1,f2,f3,f4,c0,c1,c2,total_discount,price_per_qty,new
0,1,13,27,1053,0,0,5,2,1,1,208,0.402922,71.24,-3.56,0.0,0,0,0,1,0,1,4.83871,0,1,0,0,0,1,0,0,0.475936,176.808516,356.20
1,2,13,116,48,0,0,3,1,1,1,244,0.046056,89.05,-7.12,0.0,0,0,1,0,0,1,4.83871,1,0,1,0,0,1,0,0,0.740356,1933.508559,267.15
2,6,9,635,205,0,0,7,1,1,1,533,0.004825,99.38,0.00,0.0,0,0,0,1,0,0,0.00000,1,0,1,0,0,1,0,0,0.000000,20595.627655,695.66
3,7,13,644,1050,0,0,5,1,1,1,216,0.000833,71.24,-1.42,0.0,0,0,0,0,0,1,4.83871,0,1,0,0,0,1,0,0,0.195431,85535.580477,356.20
4,9,8,1017,1489,0,0,3,1,1,1,327,0.530954,71.24,-0.18,0.0,0,0,0,1,0,1,5.16129,1,0,1,0,0,1,0,0,0.025203,134.173460,213.72


In [0]:
test = function(test, campaign_data, customer_demographics, customer_transaction_data, item_data, coupon_item_mapping)

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [0]:
test.head()

,id,campaign_id,coupon_id,customer_id,rented,income_bracket,"(brand, nunique)","(brand_type, nunique)","(category, nunique)",item_id,quantity,selling_price,other_discount,coupon_discount,18-25,26-35,36-45,46-55,56-70,X,duration,Married,f1,f2,f3,f4,c0,c1,c2,total_discount,price_per_qty,new
0,3,22,869,967,0,5,1,1,1,658,0.063135,99.20,0.00,0.0,0,0,1,0,0,1,0.0,0,1,0,0,0,1,0,0,0.000000,1571.246959,496.00
1,4,20,389,1566,0,9,3,1,1,1214,1.078453,71.24,0.00,0.0,0,1,0,0,0,0,10.0,1,0,1,0,0,1,0,0,0.000000,66.057581,641.16
2,5,22,981,510,0,1,1,1,1,1019,0.692975,71.24,-4.63,0.0,0,1,0,0,0,1,0.0,1,0,1,0,0,1,0,0,0.610254,102.803136,71.24
3,8,25,1069,361,0,3,1,1,1,323,0.084346,60.20,0.00,0.0,1,0,0,0,0,0,0.0,0,1,0,0,0,1,0,0,0.000000,713.724623,180.60
4,10,17,498,811,0,5,1,1,1,720,0.006151,70.53,0.00,0.0,0,0,0,0,0,0,0.0,0,1,0,0,0,1,0,0,0.000000,11466.272376,352.65


In [0]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,rented,income_bracket,"(brand, nunique)","(brand_type, nunique)","(category, nunique)",item_id,quantity,selling_price,other_discount,coupon_discount,18-25,26-35,36-45,46-55,56-70,X,duration,Married,f1,f2,f3,f4,c0,c1,c2,total_discount,price_per_qty,new
0,1,13,27,1053,0,0,5,2,1,1,208,0.402922,71.24,-3.56,0.0,0,0,0,1,0,1,4.83871,0,1,0,0,0,1,0,0,0.475936,176.808516,356.20
1,2,13,116,48,0,0,3,1,1,1,244,0.046056,89.05,-7.12,0.0,0,0,1,0,0,1,4.83871,1,0,1,0,0,1,0,0,0.740356,1933.508559,267.15
2,6,9,635,205,0,0,7,1,1,1,533,0.004825,99.38,0.00,0.0,0,0,0,1,0,0,0.00000,1,0,1,0,0,1,0,0,0.000000,20595.627655,695.66
3,7,13,644,1050,0,0,5,1,1,1,216,0.000833,71.24,-1.42,0.0,0,0,0,0,0,1,4.83871,0,1,0,0,0,1,0,0,0.195431,85535.580477,356.20
4,9,8,1017,1489,0,0,3,1,1,1,327,0.530954,71.24,-0.18,0.0,0,0,0,1,0,1,5.16129,1,0,1,0,0,1,0,0,0.025203,134.173460,213.72


**XG-BOOST with randomisedSearchCV**

In [0]:
from sklearn.linear_model import LogisticRegression
import xgboost as xgb


X = train.drop(["redemption_status"], axis = 1)
y = train["redemption_status"]
# sm = SMOTE(random_state = 2)
# X, y = sm.fit_sample(X, y)
# data_dmatrix = xgb.DMatrix(data=X,label=y)
# params={
#  "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
#  "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
#  "min_child_weight" : [ 1, 3, 5, 7 ],
#  "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
#  "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
# }


# xg_reg = xgb.XGBClassifier()
# random_search=RandomizedSearchCV(xg_reg,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
# # xg_reg.fit(X,y)
# random_search.fit(X,y)
# random_search.best_estimator_


In [0]:
# xg_reg = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.5, gamma=0.1,
#               learning_rate=0.3, max_delta_step=0, max_depth=8,
#               min_child_weight=7, missing=None, n_estimators=100, n_jobs=1,
#               nthread=None, objective='binary:logistic', random_state=0,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=None, subsample=1, verbosity=1)
xg_reg = xgb.XGBClassifier(scale_pos_weight=1,
                    learning_rate=0.2,  
                    colsample_bytree = 0.5,
                    subsample = 0.9,
                    objective='binary:logistic', 
                    n_estimators=100, 
                    reg_alpha = 0.3,
                    max_depth=8,
                    early_stopping_rounds=10, 
                    gamma=6)
xg_reg.fit(X,y)
preds = xg_reg.predict_proba(test)

In [0]:
preds

array([[0.99197066, 0.00802937],
       [0.997004  , 0.00299604],
       [0.76868516, 0.23131482],
       ...,
       [0.9824203 , 0.01757967],
       [0.9892289 , 0.01077112],
       [0.9802366 , 0.0197634 ]], dtype=float32)

In [0]:
sool = pd.DataFrame()
sool["id"] = test["id"]
sool["redemption_status"] = preds[:,1]
sool.set_index("id", inplace = True)
sool.to_csv("correct35.csv")



In [0]:
train.head()


,id,campaign_id,coupon_id,customer_id,redemption_status,rented,income_bracket,"(brand, nunique)","(brand_type, nunique)","(category, nunique)",item_id,quantity,selling_price,other_discount,coupon_discount,18-25,26-35,36-45,46-55,56-70,X,duration,Married,f1,f2,f3,f4,c0,c1,c2,total_discount,price_per_qty,new
0,1,13,27,1053,0,0,5,2,1,1,208,0.402922,71.24,-3.56,0.0,0,0,0,1,0,1,4.83871,0,1,0,0,0,1,0,0,0.475936,176.808516,356.20
1,2,13,116,48,0,0,3,1,1,1,244,0.046056,89.05,-7.12,0.0,0,0,1,0,0,1,4.83871,1,0,1,0,0,1,0,0,0.740356,1933.508559,267.15
2,6,9,635,205,0,0,7,1,1,1,533,0.004825,99.38,0.00,0.0,0,0,0,1,0,0,0.00000,1,0,1,0,0,1,0,0,0.000000,20595.627655,695.66
3,7,13,644,1050,0,0,5,1,1,1,216,0.000833,71.24,-1.42,0.0,0,0,0,0,0,1,4.83871,0,1,0,0,0,1,0,0,0.195431,85535.580477,356.20
4,9,8,1017,1489,0,0,3,1,1,1,327,0.530954,71.24,-0.18,0.0,0,0,0,1,0,1,5.16129,1,0,1,0,0,1,0,0,0.025203,134.173460,213.72


In [0]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,rented,"(brand, nunique)","(brand_type, nunique)","(category, nunique)",item_id,quantity,selling_price,other_discount,coupon_discount,18-25,26-35,36-45,46-55,56-70,X,duration,Married,f1,f2,f3,f4,c0,c1,c2,i1,i2,i3,i4,i5,i6,i7,i8,i9,i10,i11,total_discount,price_per_qty
0,1,13,27,1053,0,0,2,1,1,208,0.402922,71.24,-3.56,0.0,0,0,0,1,0,1,4.83871,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0.475936,176.808516
1,2,13,116,48,0,0,1,1,1,244,0.046056,89.05,-7.12,0.0,0,0,1,0,0,1,4.83871,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.740356,1933.508559
2,6,9,635,205,0,0,1,1,1,533,0.004825,99.38,0.00,0.0,0,0,0,1,0,0,0.00000,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0.000000,20595.627655
3,7,13,644,1050,0,0,1,1,1,216,0.000833,71.24,-1.42,0.0,0,0,0,0,0,1,4.83871,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0.195431,85535.580477
4,9,8,1017,1489,0,0,1,1,1,327,0.530954,71.24,-0.18,0.0,0,0,0,1,0,1,5.16129,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.025203,134.173460
